In [77]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [78]:
info = pd.read_excel("05. Rasgos_Bombuscaro_Cajanuma.xlsx") #len 83

### add rasgos 
info["mean_toughness_(kN/m)"]= info["leaf 1, punch 1"]+info["leaf 1, punch 2"]+info["leaf 2, punch 1"]+info["leaf 2, punch 2"]+info["leaf 3, punch 1"]+info["leaf 3, punch 2"] #HELP: Units
info["mean_thickness_(mm)"]= info["leaf 1, thickn 1"]+info["leaf 1, thickn 2"]+info["leaf 2, thickn 1"]+info["leaf 2, thickn 2"]+info["leaf 3, thickn 1"]+info["leaf 3, thickn 2"]
info['area_basal_m2'] = (np.pi*(info['dbh_2024']**2))/40000
info['water_content'] = (info["Leaf fresh weight (g)"] - info["Leaf dry weight (g)"])/info["Leaf fresh weight (g)"] 
info["name"] = info["genus"]+" "+info["specie"]

info = info[["Site","Sub","Plot","TreeID","name","genus","specie","dbh_2024","height_2024","mean_toughness_(kN/m)","mean_thickness_(mm)","Leaf fresh weight (g)","Leaf dry weight (g)","Leaf area (cm²)",'area_basal_m2']]
info["SLA (cm²/g)"] =   info["Leaf area (cm²)"]/info["Leaf dry weight (g)"]  #HELP: Numero de hojas   
length = len(info)


In [79]:
O18 = pd.read_excel("Andrea-cellulose_18O-20250428.xlsx") #contiene datos de Cajanuma y Bombuscaro

#rename. few spelling mistakes
O18['sample-ID'] = O18['sample-ID'].replace({'p06-8401': 'p06-8402',"p11-19003":"p01-19003"})
O18[['plot','sample']] = O18['sample-ID'].str.split('-',n=1,expand=True)
O18["Plot"] = O18["plot"].str.replace("p", "", regex=False).astype(int) #len 84
O18["sample"] = O18["sample"].astype(int) #len 84
len(O18) #84 p08-1925 duplicated HELP: obtain mean?
O18 = O18.rename(columns={"Ꟙ18O in ‰ vs DeltaSMOW":"δ18O in ‰ vs DeltaSMOW"})



In [80]:
C13 = pd.read_excel("NUMEX_cellulose_C13-10042025_results.xlsx",sheet_name="results")
#rename. few spelling mistakes
C13['sample-ID'] = C13['sample-ID'].replace({'p12-14014': 'p06-8402',
                                             'p14-1907': 'p14-19017',
                                             'p11-19003': 'p01-19003',
                                             'p12-1913': 'p12-19013',
                                             'p6-8401': 'p06-8402',
                                             "p2-2009": "p06-2009",
                                             "p11-19003":"p01-19003"})
# delete 3 rows from Galeras
C13[['plot','sample']] = C13['sample-ID'].str.split('-',n=1,expand=True)
C13["Plot"] = C13["plot"].str.replace("p", "", regex=False).astype(int) #len 84
C13["sample"] = C13["sample"].astype(int) #len 84
C13 = C13.rename(columns={"[C]": "[C]_cellulose", "Ꟙ13C (in ‰ v PDB)":"δ13C (‰ v.s.V-PDB)"})

C13

,Unnamed: 0,Unnamed: 1,sample-ID,[C]_cellulose,δ13C (‰ v.s.V-PDB),plot,sample,Plot
0,NaN,A1,p2-8933,43.91%,-28.13,p2,8933,2
1,NaN,A2,p9-1190,42.81%,-27.57,p9,1190,9
2,NaN,A3,p12-1318,42.13%,-29.59,p12,1318,12
3,NaN,A4,p6-8824,42.73%,-28.73,p6,8824,6
4,NaN,A5,p5-1014,41.85%,-27.83,p5,1014,5
...,...,...,...,...,...,...,...,...
82,NaN,G11,p10-5791,44.19%,-32.15,p10,5791,10
83,NaN,G12,p6-7208,43.55%,-30.83,p6,7208,6
84,NaN,H1,p8-1925,42.84%,-32.02,p8,1925,8
85,NaN,H2,p2-2068,42.61%,-30.99,p2,2068,2


In [81]:
bulk = pd.read_excel("Andrea_bulk CN_2025.xlsx",sheet_name="SEQ2")
bulk['sample-ID'] = bulk['sample-ID'].replace({"p15-7543":"p16-7543"})
# delete 3 rows from Galeras
bulk[['plot','sample']] = bulk['sample-ID'].str.split('-',n=1,expand=True)
bulk["Plot"] = bulk["plot"].str.replace("p", "", regex=False).astype(int) #len 84
bulk["sample"] = bulk["sample"].astype(int) #len 84
bulk = bulk.rename(columns={"[C]": "[C]_bulk", "δ13C (‰ v.s.V-PDB)": "bulk_δ13C (‰ v.s.V-PDB)"})

bulk.head(3)

,sample-ID,[N],[C]_bulk,δ15N (‰ v.s.AIR),bulk_δ13C (‰ v.s.V-PDB),plot,sample,Plot
0,p01-19003,0.015350,0.432270,0.152889,-30.631127,p01,19003,1
1,p01-5861,0.010674,0.453564,-5.350569,-29.504389,p01,5861,1
2,p02-19004,0.016347,0.433499,0.999575,-28.198170,p02,19004,2


In [82]:
merged = pd.merge(O18, info, how="left", left_on=["Plot", "sample"], right_on=["Plot", "TreeID"])
merged_2 = pd.merge(merged,C13,how="outer", right_on=["Plot","sample"], left_on=["Plot", "sample"], suffixes=('', '_drop'))

merged_3 = pd.merge(merged_2,bulk,how="outer", right_on=["Plot","sample"], left_on=["Plot", "sample"])
merged_3 = merged_3.dropna(subset=['Site']) #delete Napo sites
merged_3 = merged_3.loc[:, ~merged_3.columns.str.endswith('_drop')]

columns_keep = ["Plot","TreeID", "Sub","name","genus","specie","Site","[C]_bulk","[N]","δ15N (‰ v.s.AIR)","bulk_δ13C (‰ v.s.V-PDB)","[O]","δ18O in ‰ vs DeltaSMOW","[C]_cellulose","δ13C (‰ v.s.V-PDB)",
                "height_2024","mean_toughness_(kN/m)","mean_thickness_(mm)","Leaf fresh weight (g)","Leaf dry weight (g)","Leaf area (cm²)","SLA (cm²/g)",'area_basal_m2']


### San Francisco


In [83]:
O18SF = pd.read_excel("NUMEX_cellulose_SF_O18.xlsx")
O18SF["sample-ID"]=O18SF["sample-ID"].astype(str)
O18SF['sample-ID'] = O18SF['sample-ID'].replace({'502302':'592302'})

base = pd.read_excel("Samples_NUMEX_SF2024.xlsx")
base['samle ID'] = base['samle ID'].astype(str)
base[["Plot","TreeID"]] = base["plot/tree"].str.split("-",n=1,expand=True) #HELP: eliminate the strings for later
SF_merged = pd.merge(O18SF,base,how="left", left_on="sample-ID", right_on="samle ID")
SF_merged = SF_merged.rename(columns={"Ꟙ18O in ‰ vs DeltaSMOW":"δ18O in ‰ vs DeltaSMOW","species":"name","treatment":"Sub"})
keep = ["sample-ID","Plot","TreeID","name","Sub","[O]","δ18O in ‰ vs DeltaSMOW"]
SF = SF_merged[keep]
#wait for the other results


In [84]:
all= merged_3[columns_keep]
#all= all.drop(columns = ["Unnamed: 0"]) #clean


all.to_excel("Numex_preliminar_base.xlsx")

# clean values of C that are incorrect. Too low, less than 35%
 #iWUE = iWUE[iWUE['[C] m/m (%)'] >= 0.35]

In [85]:
test = pd.concat([all,SF], join = "outer")
test.to_excel("test.xlsx")